# Simple Entry Removal Algorithm 2 (SER 2)

In [1]:
import numpy as np

In [2]:
# This function is used in the SER 2 function.
# residue_matrix_R is a np.ndarray.
# chosen_entry_pos_each_col is a 1-by-PBN_col_sum np.ndarray.
# PBN_col_num is the number of columns present in residue_matrix_R

def choose_coefficient(residue_matrix_R, chosen_entry_pos_each_col, PBN_col_num):
    col_counter = 0
    chosen_entry_xi = residue_matrix_R[chosen_entry_pos_each_col[0, col_counter], col_counter]
    
    for col_counter in range(PBN_col_num):
        current_entry = residue_matrix_R[chosen_entry_pos_each_col[0, col_counter], col_counter]
        if current_entry < chosen_entry_xi:
            chosen_entry_xi = current_entry
    
    return chosen_entry_xi

### Testing the function choose_coefficient

In [4]:
test1 = np.array([[0.2, 0.9], 
                  [0.8, 0.1]])
choose_coefficient(test1, np.argmax(test1, axis=0, keepdims=True), 2)

0.8

In [5]:
test2 = np.array([[12, 45, 75,  3], 
                  [45,  4,  3, 61], 
                  [ 5,  5,  5, 67]])
choose_coefficient(test2, np.argmax(test2, axis=0, keepdims=True), 4)

45

In [6]:
test3 = np.array([[1, 5, 6,  0], 
                  [4, 0, 2,  0], 
                  [5, 2, 0, 10], 
                  [0, 3, 2,  0]])
choose_coefficient(test3, np.argmax(test3, axis=0, keepdims=True), 4)

5

### The SER 2 function

In [3]:
def SER_2(PBN_matrix_P):  # input: 2^n-by-2^n ndarray consisting of integers.
    PBN_row_num = PBN_matrix_P.shape[0]
    PBN_col_num = PBN_matrix_P.shape[1]
    coefficient_list_xi = []  # the coefficients of BN matrices to be output
    # the BN matrices to be output
    BN_matrices_list_Ai = np.array([[-1] * PBN_col_num])  # the first row of BN_matrices_list_Ai will be
                                                          # replaced in the first iteration of pseudocode
                                                          # steps 1 to 4.
                                                          
    residue_matrix_R = PBN_matrix_P.copy()
    k = 1  # counts the number of iterations of pseudocode Steps 1 to 4

    chosen_entry_pos_each_col = np.argmax(residue_matrix_R, axis=0, keepdims=True)
    chosen_entry_xi = choose_coefficient(residue_matrix_R, chosen_entry_pos_each_col, PBN_col_num)
    
    coefficient_list_xi.append(chosen_entry_xi)
    BN_matrices_list_Ai = chosen_entry_pos_each_col
    
    for col_counter in range(PBN_col_num):
        row_to_deduct = chosen_entry_pos_each_col[0, col_counter]
        residue_matrix_R[row_to_deduct, col_counter] -= chosen_entry_xi
    
    zero_PBN_matrix = np.zeros((PBN_row_num, PBN_col_num), dtype=int)
    
    while (not np.array_equal(residue_matrix_R, zero_PBN_matrix)):
        k += 1
        chosen_entry_pos_each_col = np.argmax(residue_matrix_R, axis=0, keepdims=True)
        chosen_entry_xi = choose_coefficient(residue_matrix_R, chosen_entry_pos_each_col, PBN_col_num)
    
        coefficient_list_xi.append(chosen_entry_xi)
        BN_matrices_list_Ai = np.append(BN_matrices_list_Ai, chosen_entry_pos_each_col, axis=0)
        
        for col_counter in range(PBN_col_num):
            row_to_deduct = chosen_entry_pos_each_col[0, col_counter]
            residue_matrix_R[row_to_deduct, col_counter] -= chosen_entry_xi
    
    return k, coefficient_list_xi, BN_matrices_list_Ai

### (2022-12-17) First time testing if the SER 2 implementation is bug-free

#### Test 1
Result: function output matches with the correct output.

In [14]:
test_PBN_matrix_1 = np.array([[2, 7], 
                              [8, 3]])

SER_2(test_PBN_matrix_1)

(3,
 [7, 2, 1],
 array([[1, 0],
        [0, 1],
        [1, 1]], dtype=int64))

#### Test 2 (Small Example PBN Matrix Mentioned in the GER paper Section 2.2)
Result: function output matches with the correct output.

In [10]:
test_PBN_matrix_2 = np.array([[1, 5, 6, 0], 
                              [4, 0, 2, 0], 
                              [5, 2, 0, 10], 
                              [0, 3, 2, 0]])
SER_2(test_PBN_matrix_2)

(4,
 [5, 2, 2, 1],
 array([[2, 0, 0, 2],
        [1, 3, 1, 2],
        [1, 2, 3, 2],
        [0, 3, 0, 2]], dtype=int64))

### Outputs of SER 2 on different small PBN matrices

#### The following function is used to permute the entries in each column of the three PBN matrices used to break SER 2.

In [6]:
def permute_entries_in_each_col(input_PBN_matrix):
    num_of_rows, num_of_cols = input_PBN_matrix.shape
    permuted_PBN_matrix = np.zeros(input_PBN_matrix.shape, dtype=int)
    
    for col_counter in range(num_of_cols):
        rearranged_row_indices = np.random.permutation(num_of_rows)
        for row_counter in range(num_of_rows):
            permuted_PBN_matrix[row_counter, col_counter] = \
            input_PBN_matrix[rearranged_row_indices[row_counter], col_counter]
    
    return permuted_PBN_matrix

#### The following function is used to permute the columns of the three PBN matrices used to break SER 2.

In [7]:
def permute_the_cols(input_PBN_matrix):
    num_of_rows, num_of_cols = input_PBN_matrix.shape
    permuted_PBN_matrix = np.zeros(input_PBN_matrix.shape, dtype=int)
    rearranged_col_indices = np.random.permutation(num_of_cols)
    
    for col_counter in range(num_of_cols):
        permuted_PBN_matrix[:, col_counter] = input_PBN_matrix[:, rearranged_col_indices[col_counter]]
    
    return permuted_PBN_matrix

#### [1] PBN Matrix from Default Data

In [11]:
test_PBN_matrix_3 = np.array([[57,  0, 10,  0,  0,  4,  0,  0], 
                              [ 0, 15,  0,  0,  0,  8,  0,  0], 
                              [ 0,  8, 40, 25, 25,  0, 67,  0], 
                              [ 0,  0,  0,  0, 38,  0,  0,  0],
                              [14, 31,  0, 50, 12, 13, 33,  6],
                              [29, 31, 20,  0, 25, 29,  0, 39],
                              [ 0, 15, 30,  0,  0, 13,  0,  0],
                              [ 0,  0,  0, 25,  0, 33,  0, 55]])
SER_2(test_PBN_matrix_3)

(11,
 [31, 25, 13, 10, 8, 4, 4, 2, 1, 1, 1],
 array([[0, 4, 2, 4, 3, 7, 2, 7],
        [5, 5, 6, 2, 2, 5, 2, 5],
        [0, 1, 5, 7, 5, 4, 4, 7],
        [4, 6, 0, 4, 4, 6, 4, 5],
        [0, 2, 2, 7, 5, 1, 2, 7],
        [0, 5, 5, 4, 3, 0, 4, 4],
        [4, 6, 6, 4, 5, 5, 4, 5],
        [5, 1, 5, 7, 3, 6, 2, 7],
        [5, 5, 2, 7, 4, 7, 4, 4],
        [0, 5, 5, 4, 3, 6, 2, 4],
        [5, 6, 6, 7, 4, 7, 4, 7]], dtype=int64))

#### [2] Nursing Home PBN Matrix

In [13]:
test_PBN_matrix_4 = np.array([[0, 6, 8, 1, 7, 5, 8, 5, 6, 5, 3, 4, 10, 2, 2, 0], 
                              [8, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,  0, 0, 0, 3], 
                              [2, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,  0, 0, 0, 0], 
                              [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  0, 0, 0, 0],
                              [0, 1, 1, 8, 2, 4, 2, 5, 0, 3, 2, 3,  0, 3, 4, 1],
                              [0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,  0, 0, 0, 0],
                              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  0, 0, 0, 0],
                              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  0, 0, 0, 0],
                              [0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 3,  0, 0, 2, 1],
                              [0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 0,  0, 0, 0, 0],
                              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  0, 0, 0, 0],
                              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  0, 0, 0, 0],
                              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0,  0, 4, 2, 5],
                              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  0, 1, 0, 0],
                              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  0, 0, 0, 0],
                              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  0, 0, 0, 0]
                             ])
SER_2(test_PBN_matrix_4)

(7,
 [3, 2, 1, 1, 1, 1, 1],
 array([[ 1,  0,  0,  4,  0,  0,  0,  0,  0,  0,  0,  0,  0, 12,  4, 12],
        [ 1,  0,  0,  4,  0,  4,  0,  4,  0,  4, 12,  4,  0,  4,  0,  1],
        [ 1,  0,  0,  4,  0,  0,  0,  4,  0,  0,  4,  8,  0,  0,  8, 12],
        [ 1,  1,  0,  4,  4,  4,  0,  0,  1,  9,  9,  8,  0,  0, 12,  1],
        [ 2,  2,  0,  0,  0,  0,  4,  4,  2,  0,  4,  0,  0,  4,  4,  4],
        [ 1,  3,  1,  4,  4,  4,  0,  0,  8,  4,  9,  4,  0, 12,  8,  8],
        [ 2,  4,  4,  5,  8,  5,  4,  4,  9,  9, 12,  8,  0, 13, 12, 12]],
       dtype=int64))

#### [3] First PBN matrix to break SER 2

Remark: the sum of the entries in each column of this matrix equals 110.

##### Execute SER 2 before permuting the PBN matrix.

In [5]:
x1 = 10
x2 = 12
x3 = 15
x4 = 19
x5 = 24
x6 = 30

test_PBN_matrix_5 = np.array([[   x1,    x1,    x1,    x1,    x1, x1+x2, x1+x2, x1+x2], 
                              [   x2,    x2,    x2,    x2,    x2,     0,     0,     0], 
                              [x3+x4, x3+x4,    x3,    x3,    x3,    x3,    x3, x3+x4], 
                              [    0,     0,    x4,    x4,    x4,    x4,    x4,     0],
                              [   x5, x5+x6, x5+x6, x5+x6, x5+x6,    x5,    x5,    x5],
                              [   x6,     0,     0,     0,     0,    x6,    x6,    x6],
                              [    0,     0,     0,     0,     0,     0,     0,     0],
                              [    0,     0,     0,     0,     0,     0,     0,     0]])

test_PBN_matrix_5

array([[10, 10, 10, 10, 10, 22, 22, 22],
       [12, 12, 12, 12, 12,  0,  0,  0],
       [34, 34, 15, 15, 15, 15, 15, 34],
       [ 0,  0, 19, 19, 19, 19, 19,  0],
       [24, 54, 54, 54, 54, 24, 24, 24],
       [30,  0,  0,  0,  0, 30, 30, 30],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0]])

In [9]:
SER_2(test_PBN_matrix_5)

(10,
 [30, 24, 19, 12, 10, 6, 4, 3, 1, 1],
 array([[2, 4, 4, 4, 4, 5, 5, 2],
        [5, 2, 4, 4, 4, 4, 4, 5],
        [4, 4, 3, 3, 3, 0, 0, 4],
        [1, 1, 2, 2, 2, 3, 3, 0],
        [0, 0, 1, 1, 1, 2, 2, 0],
        [5, 2, 0, 0, 0, 3, 3, 5],
        [4, 4, 0, 0, 0, 2, 2, 4],
        [2, 2, 2, 2, 2, 0, 0, 2],
        [2, 2, 1, 1, 1, 2, 2, 2],
        [4, 4, 1, 1, 1, 3, 3, 4]], dtype=int64))

##### Let's permute the entries in the PBN matrix.

In [10]:
# test_PBN_matrix_5_permuted = permute_the_cols(permute_entries_in_each_col(test_PBN_matrix_5))
test_PBN_matrix_5_permuted = np.array([[12, 30, 22, 10, 10, 15, 54, 34],
                                       [10, 24, 19, 54, 30,  0,  0,  0],
                                       [54, 15,  0, 12, 12,  0,  0, 30],
                                       [ 0,  0, 24, 15, 24, 19, 10,  0],
                                       [ 0,  0,  0,  0, 34, 10, 12,  0],
                                       [19,  0, 15,  0,  0, 12,  0, 22],
                                       [15, 19,  0, 19,  0, 54,  0, 24],
                                       [ 0, 22, 30,  0,  0,  0, 34,  0]])

##### Execute SER 2 on the permuted PBN matrix.

In [13]:
SER_2(test_PBN_matrix_5_permuted)

(10,
 [30, 24, 19, 12, 10, 6, 4, 3, 1, 1],
 array([[2, 0, 7, 1, 4, 6, 0, 0],
        [2, 1, 3, 1, 1, 6, 7, 2],
        [5, 7, 0, 6, 3, 3, 0, 6],
        [6, 6, 1, 3, 2, 0, 4, 5],
        [0, 2, 5, 2, 0, 5, 3, 5],
        [1, 6, 1, 0, 1, 4, 7, 2],
        [1, 2, 5, 0, 3, 4, 0, 6],
        [6, 7, 0, 3, 4, 0, 7, 0],
        [0, 2, 1, 2, 3, 5, 0, 0],
        [0, 6, 5, 2, 4, 5, 7, 6]], dtype=int64))

#### [4] Second PBN matrix to break SER 2

Remark: the sum of the entries in each column of this matrix equals 110.

##### Execute SER 2 before permuting the PBN matrix.

In [14]:
x1 = 10
x2 = 12
x3 = 15
x4 = 19
x5 = 24
x6 = 30

test_PBN_matrix_6 = np.array([[   x1,    x1, x1+x2, x1+x2, x1+x2, x1+x3, x1+x3, x1+x3], 
                              [   x2,    x2,    x3,    x3,    x3,    x2,    x2,    x2], 
                              [   x3,    x3,     0,     0,     0,     0,     0,     0], 
                              [x4+x6, x4+x6, x4+x6, x4+x5, x4+x5, x4+x5,    x4,    x4],
                              [   x5,    x5,    x5,    x6,    x6,    x6,    x5,    x5],
                              [    0,     0,     0,     0,     0,     0,    x6,    x6],
                              [    0,     0,     0,     0,     0,     0,     0,     0],
                              [    0,     0,     0,     0,     0,     0,     0,     0]])

test_PBN_matrix_6

array([[10, 10, 22, 22, 22, 25, 25, 25],
       [12, 12, 15, 15, 15, 12, 12, 12],
       [15, 15,  0,  0,  0,  0,  0,  0],
       [49, 49, 49, 43, 43, 43, 19, 19],
       [24, 24, 24, 30, 30, 30, 24, 24],
       [ 0,  0,  0,  0,  0,  0, 30, 30],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0]])

In [16]:
SER_2(test_PBN_matrix_6)

(9,
 [30, 24, 19, 13, 12, 6, 3, 2, 1],
 array([[3, 3, 3, 3, 3, 3, 5, 5],
        [4, 4, 4, 4, 4, 4, 0, 0],
        [3, 3, 0, 0, 0, 0, 4, 4],
        [2, 2, 3, 1, 1, 3, 3, 3],
        [1, 1, 1, 3, 3, 1, 1, 1],
        [0, 0, 3, 4, 4, 0, 3, 3],
        [0, 0, 0, 0, 0, 4, 4, 4],
        [2, 2, 1, 1, 1, 4, 4, 4],
        [0, 0, 1, 3, 3, 4, 0, 0]], dtype=int64))

##### Let's permute the entries in the PBN matrix.

In [18]:
# test_PBN_matrix_6_permuted = permute_the_cols(permute_entries_in_each_col(test_PBN_matrix_6))
test_PBN_matrix_6_permuted = np.array([[ 0,  0,  0, 49,  0, 43,  0, 49],
                                       [ 0, 30, 12,  0, 30,  0, 25,  0],
                                       [25,  0,  0, 15,  0, 22, 12, 15],
                                       [ 0,  0, 10, 24, 19,  0, 30,  0],
                                       [30, 43, 15,  0, 24, 15,  0,  0],
                                       [43,  0, 49,  0,  0,  0, 19, 24],
                                       [ 0, 22,  0, 22, 12, 30,  0, 12],
                                       [12, 15, 24,  0, 25,  0, 24, 10]])

##### Execute SER 2 on the permuted PBN matrix.

In [20]:
SER_2(test_PBN_matrix_6_permuted)

(9,
 [30, 24, 19, 13, 12, 6, 3, 2, 1],
 array([[5, 4, 5, 0, 1, 0, 3, 0],
        [4, 1, 7, 3, 7, 6, 1, 5],
        [2, 6, 5, 6, 4, 2, 7, 0],
        [5, 7, 4, 0, 3, 4, 5, 2],
        [7, 4, 1, 2, 6, 0, 2, 6],
        [2, 1, 3, 0, 3, 6, 5, 7],
        [4, 6, 3, 2, 4, 2, 7, 7],
        [4, 7, 4, 6, 4, 4, 7, 2],
        [4, 4, 3, 6, 7, 0, 1, 7]], dtype=int64))

#### [5] Third PBN matrix to break SER 2

Remark: the sum of the entries in each column of this matrix equals 265.

##### Execute SER 2 before permuting the PBN matrix.

In [21]:
x1 = 37
x2 = 26
x3 = 17
x4 = 9
x5 = 59
x6 = 49
x7 = 39
x8 = 29

test_PBN_matrix_7 = np.array([[   x1,    x1,    x1,    x1, x1+x2, x1+x2, x1+x2, x1+x2, x1+x3, x1+x3, x1+x3, x1+x3, x1+x4, x1+x4, x1+x4, x1+x4], 
                              [   x2,    x2,    x2,    x2,    x3,    x3,    x3,    x3,    x2,    x2,    x2,    x2,    x2,    x2,    x2,    x2], 
                              [   x3,    x3,    x3,    x3,    x4,    x4,    x4,    x4,    x4,    x4,    x4,    x4,    x3,    x3,    x3,    x3], 
                              [   x4,    x4,    x4,    x4,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
                              [    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
                              [    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
                              [x5+x6, x5+x6, x5+x7, x5+x8, x5+x7, x5+x8, x5+x6,    x5, x5+x7, x5+x8, x5+x6,    x5, x5+x7, x5+x8,    x5,    x5],
                              [    0,     0,    x6,    x6,    x6,    x6,     0,    x6,    x6,    x6,     0,    x6,    x6,    x6,    x6,    x6],
                              [   x7,    x7,     0,    x7,     0,    x7,    x7,    x7,     0,    x7,    x7,    x7,     0,    x7,    x7,    x7],
                              [   x8,    x8,    x8,     0,    x8,     0,    x8,    x8,    x8,     0,    x8,    x8,    x8,     0,    x8,    x8],
                              [    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
                              [    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
                              [    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
                              [    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
                              [    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
                              [    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
                             ])
test_PBN_matrix_7

array([[ 37,  37,  37,  37,  63,  63,  63,  63,  54,  54,  54,  54,  46,
         46,  46,  46],
       [ 26,  26,  26,  26,  17,  17,  17,  17,  26,  26,  26,  26,  26,
         26,  26,  26],
       [ 17,  17,  17,  17,   9,   9,   9,   9,   9,   9,   9,   9,  17,
         17,  17,  17],
       [  9,   9,   9,   9,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0],
       [108, 108,  98,  88,  98,  88, 108,  59,  98,  88, 108,  59,  98,
         88,  59,  59],
       [  0,   0,  49,  49,  49,  49,   0,  49,  49,  49,   0,  49,  49,
         49,  49,  49],
       [ 39,  39,   0,  39,   0,  39,  39,  39,   0,  39,  39,  39,   0,
         39,  39,  39],
       [ 29,  29,  29,   0,  29,   0,  29,  29,  29,   0,  29,  29,  29,
          0,  29,  29],
       [  0,   0,   0,   0,   

In [23]:
SER_2(test_PBN_matrix_7)

(13,
 [59, 49, 39, 37, 29, 17, 10, 9, 7, 4, 2, 2, 1],
 array([[6, 6, 6, 6, 6, 6, 6, 0, 6, 6, 6, 6, 6, 6, 6, 6],
        [6, 6, 7, 7, 0, 0, 0, 6, 0, 0, 0, 0, 7, 7, 7, 7],
        [8, 8, 6, 8, 7, 7, 6, 7, 7, 7, 6, 7, 0, 0, 0, 0],
        [0, 0, 0, 0, 6, 8, 8, 8, 6, 8, 8, 8, 6, 8, 8, 8],
        [9, 9, 9, 6, 9, 6, 9, 9, 9, 6, 9, 9, 9, 6, 9, 9],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [2, 2, 2, 2, 0, 0, 0, 6, 7, 7, 6, 7, 2, 2, 2, 2],
        [1, 1, 1, 1, 7, 7, 6, 7, 1, 1, 1, 1, 1, 1, 1, 1],
        [3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0],
        [2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2],
        [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
        [3, 3, 3, 3, 6, 8, 8, 8, 6, 8, 8, 8, 6, 8, 8, 8],
        [2, 2, 2, 2, 7, 7, 6, 7, 0, 0, 0, 0, 2, 2, 2, 2]], dtype=int64))

##### Let's permute the entries in the PBN matrix.

In [24]:
# test_PBN_matrix_7_permuted = permute_the_cols(permute_entries_in_each_col(test_PBN_matrix_7))
test_PBN_matrix_7_permuted = np.array([[ 26,   0,   0,   0,  59,   0,  49,   0,  29,   0,   0,   0,   0,   9,   0,  46],
                                       [  0,   0,  39,  17,   0,   0,   0,  49,  49,   0,   0,   0,   0,  98,  54,  17],
                                       [  0,   0,  26,  49,   0,   0,   0,   9,   0,   0,   0,  59,   9,   0,   0,   0],
                                       [  0,   0,   0,   0,   0,   9,  54,   0,   0,   0,   0,   0,  39,   0,  49,  26],
                                       [ 49,   0,   9,   0,   0,  29,   0,   0,   0,   0, 108,  63,   0,  17,  59,   0],
                                       [  0,   0,   0,  26,   0,   0,   0,  17,  39,  29,   0,   0,   0,   0,   0,  88],
                                       [ 17,  63,  88,   0,   0,   0,   0,  98,   0,   9,  37,   0,  88,   0,   0,  49],
                                       [  0,   0,   0,   0,  49,   0,   0,   0,  17,  37,   9,  29,  63,  49,  39,   0],
                                       [ 98,   0,   0,   0,  46, 108,  26,   0,   0,   0,   0,   9,   0,  63,   9,   0],
                                       [  0,  29,   0,   0,  17,   0,  29,   0,  46,   0,  26,   0,   0,   0,  26,   0],
                                       [ 29, 108,   0,  88,   0,  39,   0,   0,   0,   0,  39,   0,   0,  29,   0,   0],
                                       [  0,  39,   0,  39,   0,   0,   0,   0,  26,  26,  29,  39,   0,   0,   0,   0],
                                       [  0,   0,   0,   0,  29,   0,  98,  29,   0,  39,   0,   0,   0,   0,   0,   0],
                                       [ 46,   0,  49,  37,  39,  26,   9,   0,   0, 108,  17,   0,  49,   0,  29,   0],
                                       [  0,   9,  54,   9,   0,  54,   0,  37,   0,  17,   0,  49,  17,   0,   0,  39],
                                       [  0,  17,   0,   0,  26,   0,   0,  26,  59,   0,   0,  17,   0,   0,   0,   0]])


##### Execute SER 2 on the permuted PBN matrix.

In [26]:
SER_2(test_PBN_matrix_7_permuted)

(13,
 [59, 49, 39, 37, 29, 17, 10, 9, 7, 4, 2, 2, 1],
 array([[ 8, 10,  6, 10,  0,  8, 12,  6, 15, 13,  4,  4,  6,  1,  4,  5],
        [ 4,  6, 14,  2,  7, 14,  3,  1,  1, 13,  4,  2,  7,  8,  1,  6],
        [13, 10, 13, 11,  8,  8,  0,  6,  9, 12, 10, 14, 13,  7,  3,  0],
        [ 8, 11,  1, 13, 13, 10, 12, 14,  5,  7,  6, 11,  3,  1,  7, 14],
        [10,  9,  6, 10, 12,  4,  9, 12,  0,  5, 11,  7,  6, 10, 13,  5],
        [ 0, 15,  2,  5, 15, 13,  8, 15, 11, 11,  9, 15, 14,  4,  9,  3],
        [ 6,  6, 13,  1,  9,  8,  0,  5,  7, 14, 13,  2,  7,  8,  3,  1],
        [ 0, 10,  2,  5, 15,  3,  8,  2, 11,  6,  7, 14, 13,  7,  8,  3],
        [ 6, 14,  4, 14,  8, 13, 13, 15,  7, 11,  9,  8,  2,  0,  9,  0],
        [13,  6, 14,  1,  9, 14,  3,  5,  9, 14, 13,  4,  7,  8,  1,  1],
        [13, 11,  1,  1,  9, 10, 12,  5,  9, 14, 13,  8,  2,  0,  7,  1],
        [ 8, 14,  4, 14, 13, 13, 13, 15,  5, 11,  9, 11,  3,  1,  9, 14],
        [13, 10, 14,  1,  9, 14,  3,  5,  9, 14, 13, 14, 1

#### [6] Small Example PBN Matrix Mentioned in the GER paper Section 2.2

In [5]:
test_PBN_matrix_2 = np.array([[1, 5, 6, 0], 
                              [4, 0, 2, 0], 
                              [5, 2, 0, 10], 
                              [0, 3, 2, 0]])
SER_2(test_PBN_matrix_2)

(4,
 [5, 2, 2, 1],
 array([[2, 0, 0, 2],
        [1, 3, 1, 2],
        [1, 2, 3, 2],
        [0, 3, 0, 2]], dtype=int64))

# Testing SER 2 on the three PBN matrices from the MOMP paper.

### 1. First PBN matrix $P_1$

It can be proved that SER 2 finds the sparsest possible decomposition for $P_1$.

In [4]:
MOMP_PBN_P1 = np.array([[1, 3, 2, 1], 
                        [2, 3, 2, 0], 
                        [0, 0, 6, 4], 
                        [7, 4, 0, 5]])
SER_2(MOMP_PBN_P1)

(5,
 [4, 2, 2, 1, 1],
 array([[3, 3, 2, 3],
        [3, 0, 0, 2],
        [1, 1, 1, 2],
        [0, 0, 2, 0],
        [3, 1, 2, 3]], dtype=int64))

### 2. Second PBN matrix $P_2$

It can be proved that SER 2 finds the sparsest possible decomposition for $P_2$.

In [5]:
MOMP_PBN_P2 = np.array([[1, 3, 2, 1, 0, 0, 0, 0], 
                        [2, 3, 2, 0, 0, 0, 0, 0], 
                        [0, 0, 6, 4, 0, 0, 0, 0], 
                        [7, 4, 0, 5, 0, 0, 0, 0],
                        [0, 0, 0, 0, 1, 3, 2, 1],
                        [0, 0, 0, 0, 2, 3, 2, 0],
                        [0, 0, 0, 0, 0, 0, 6, 4],
                        [0, 0, 0, 0, 7, 4, 0, 5]])
SER_2(MOMP_PBN_P2)

(5,
 [4, 2, 2, 1, 1],
 array([[3, 3, 2, 3, 7, 7, 6, 7],
        [3, 0, 0, 2, 7, 4, 4, 6],
        [1, 1, 1, 2, 5, 5, 5, 6],
        [0, 0, 2, 0, 4, 4, 6, 4],
        [3, 1, 2, 3, 7, 5, 6, 7]], dtype=int64))

### 3. Third PBN matrix $P_3$

In [7]:
MOMP_PBN_P3 = np.array([[57,  0, 10,  0,  0,  4,  0,  0], 
                        [14, 31,  0, 50, 12, 13, 33,  6], 
                        [ 0,  8, 40, 25, 25,  0, 67,  0], 
                        [ 0, 15,  0,  0,  0,  8,  0,  0],
                        [ 0, 15, 30,  0,  0, 13,  0,  0],
                        [29, 31, 20,  0, 25, 29,  0, 39],
                        [ 0,  0,  0,  0, 38,  0,  0,  0],
                        [ 0,  0,  0, 25,  0, 33,  0, 55]])
SER_2(MOMP_PBN_P3)

(11,
 [31, 25, 13, 10, 8, 4, 4, 2, 1, 1, 1],
 array([[0, 1, 2, 1, 6, 7, 2, 7],
        [5, 5, 4, 2, 2, 5, 2, 5],
        [0, 3, 5, 7, 5, 1, 1, 7],
        [1, 4, 0, 1, 1, 4, 1, 5],
        [0, 2, 2, 7, 5, 3, 2, 7],
        [0, 5, 5, 1, 6, 0, 1, 1],
        [1, 4, 4, 1, 5, 5, 1, 5],
        [5, 3, 5, 7, 6, 4, 2, 7],
        [5, 5, 2, 7, 1, 7, 1, 1],
        [0, 4, 4, 1, 1, 4, 1, 1],
        [5, 5, 5, 7, 6, 7, 2, 7]], dtype=int64))

# Testing SER 2 on the PBN matrices from the projection-based gradient descent method (PBGDM) paper.

### 1. First PBN matrix $P_1$

It can be proved that SER 2 finds the sparsest possible decomposition for $P_1$.

In [9]:
PBGDM_PBN_P1 = np.array([[1, 3, 5, 6], 
                         [0, 7, 0, 0], 
                         [0, 0, 5, 0], 
                         [9, 0, 0, 4]])
SER_2(PBGDM_PBN_P1)

(4,
 [5, 3, 1, 1],
 array([[3, 1, 0, 0],
        [3, 0, 2, 3],
        [0, 1, 2, 0],
        [3, 1, 2, 3]], dtype=int64))

### 2. Second PBN matrix $P_3$

It can be proved that SER 2 finds the sparsest possible decomposition for $P_3$.

In [10]:
PBGDM_PBN_P3 = np.array([[10,  0,  0, 2,  0, 0, 0,  0], 
                         [ 0,  0,  0, 2,  0, 0, 0,  0], 
                         [ 0,  0,  0, 0, 10, 0, 0,  0], 
                         [ 0,  0,  0, 0,  0, 0, 0,  0],
                         [ 0,  0,  0, 3,  0, 0, 5,  0],
                         [ 0,  0,  0, 3,  0, 0, 5,  0],
                         [ 0, 10, 10, 0,  0, 5, 0,  0],
                         [ 0,  0,  0, 0,  0, 5, 0, 10]])
SER_2(PBGDM_PBN_P3)

(4,
 [3, 3, 2, 2],
 array([[0, 6, 6, 4, 2, 6, 4, 7],
        [0, 6, 6, 5, 2, 7, 5, 7],
        [0, 6, 6, 0, 2, 6, 4, 7],
        [0, 6, 6, 1, 2, 7, 5, 7]], dtype=int64))

### 3. Third PBN matrix $P_4(d)$, where $d = 0.01, 0.02, 0.03, 0.04$.

It can be proved that SER 2 finds the sparsest possible decomposition for $P_4(0.01)$.

In [11]:
d = 1
PBGDM_PBN_P4_d1 = np.array([[100 - d,       0,       0, 20,       0,  0,  0,       0], 
                            [      0,       0,       0, 20,       0,  0,  0,       0], 
                            [      0,       0,       0,  0, 100 - d,  0,  0,       0], 
                            [      d,       d,       d,  0,       d,  0,  0,       d],
                            [      0,       0,       0, 30,       0,  0, 50,       0],
                            [      0,       0,       0, 30,       0,  0, 50,       0],
                            [      0, 100 - d, 100 - d,  0,       0, 50,  0,       0],
                            [      0,       0,       0,  0,       0, 50,  0, 100 - d]])
SER_2(PBGDM_PBN_P4_d1)

(5,
 [30, 30, 20, 19, 1],
 array([[0, 6, 6, 4, 2, 6, 4, 7],
        [0, 6, 6, 5, 2, 7, 5, 7],
        [0, 6, 6, 0, 2, 6, 4, 7],
        [0, 6, 6, 1, 2, 7, 5, 7],
        [3, 3, 3, 1, 3, 7, 5, 3]], dtype=int64))

It can be proved that SER 2 finds the sparsest possible decomposition for $P_4(0.02)$.

In [12]:
d = 1
PBGDM_PBN_P4_d2 = np.array([[50 - d,      0,      0, 10,      0,  0,  0,      0], 
                            [     0,      0,      0, 10,      0,  0,  0,      0], 
                            [     0,      0,      0,  0, 50 - d,  0,  0,      0], 
                            [     d,      d,      d,  0,      d,  0,  0,      d],
                            [     0,      0,      0, 15,      0,  0, 25,      0],
                            [     0,      0,      0, 15,      0,  0, 25,      0],
                            [     0, 50 - d, 50 - d,  0,      0, 25,  0,      0],
                            [     0,      0,      0,  0,      0, 25,  0, 50 - d]])
SER_2(PBGDM_PBN_P4_d2)

(5,
 [15, 15, 10, 9, 1],
 array([[0, 6, 6, 4, 2, 6, 4, 7],
        [0, 6, 6, 5, 2, 7, 5, 7],
        [0, 6, 6, 0, 2, 6, 4, 7],
        [0, 6, 6, 1, 2, 7, 5, 7],
        [3, 3, 3, 1, 3, 7, 5, 3]], dtype=int64))

It can be proved that SER 2 finds the sparsest possible decomposition for $P_4(0.03)$.

In [13]:
d = 3
PBGDM_PBN_P4_d3 = np.array([[100 - d,       0,       0, 20,       0,  0,  0,       0], 
                            [      0,       0,       0, 20,       0,  0,  0,       0], 
                            [      0,       0,       0,  0, 100 - d,  0,  0,       0], 
                            [      d,       d,       d,  0,       d,  0,  0,       d],
                            [      0,       0,       0, 30,       0,  0, 50,       0],
                            [      0,       0,       0, 30,       0,  0, 50,       0],
                            [      0, 100 - d, 100 - d,  0,       0, 50,  0,       0],
                            [      0,       0,       0,  0,       0, 50,  0, 100 - d]])
SER_2(PBGDM_PBN_P4_d3)

(5,
 [30, 30, 20, 17, 3],
 array([[0, 6, 6, 4, 2, 6, 4, 7],
        [0, 6, 6, 5, 2, 7, 5, 7],
        [0, 6, 6, 0, 2, 6, 4, 7],
        [0, 6, 6, 1, 2, 7, 5, 7],
        [3, 3, 3, 1, 3, 7, 5, 3]], dtype=int64))

It can be proved that SER 2 finds the sparsest possible decomposition for $P_4(0.04)$.

In [14]:
d = 2
PBGDM_PBN_P4_d4 = np.array([[50 - d,      0,      0, 10,      0,  0,  0,      0], 
                            [     0,      0,      0, 10,      0,  0,  0,      0], 
                            [     0,      0,      0,  0, 50 - d,  0,  0,      0], 
                            [     d,      d,      d,  0,      d,  0,  0,      d],
                            [     0,      0,      0, 15,      0,  0, 25,      0],
                            [     0,      0,      0, 15,      0,  0, 25,      0],
                            [     0, 50 - d, 50 - d,  0,      0, 25,  0,      0],
                            [     0,      0,      0,  0,      0, 25,  0, 50 - d]])
SER_2(PBGDM_PBN_P4_d4)

(5,
 [15, 15, 10, 8, 2],
 array([[0, 6, 6, 4, 2, 6, 4, 7],
        [0, 6, 6, 5, 2, 7, 5, 7],
        [0, 6, 6, 0, 2, 6, 4, 7],
        [0, 6, 6, 1, 2, 7, 5, 7],
        [3, 3, 3, 1, 3, 7, 5, 3]], dtype=int64))

### 4. Fourth PBN matrix $P_6(d)$, where $d = 0.01, 0.02, 0.03, 0.04$.

It can be proved that SER 2 finds the sparsest possible decomposition for $P_6(0.01)$.

In [15]:
PBGDM_PBN_P6_d1 = np.zeros((16, 16), dtype=int)
PBGDM_PBN_P6_d1[0:8, 0:8] = PBGDM_PBN_P4_d1
PBGDM_PBN_P6_d1[8:16, 8:16] = PBGDM_PBN_P4_d1

SER_2(PBGDM_PBN_P6_d1)

(5,
 [30, 30, 20, 19, 1],
 array([[ 0,  6,  6,  4,  2,  6,  4,  7,  8, 14, 14, 12, 10, 14, 12, 15],
        [ 0,  6,  6,  5,  2,  7,  5,  7,  8, 14, 14, 13, 10, 15, 13, 15],
        [ 0,  6,  6,  0,  2,  6,  4,  7,  8, 14, 14,  8, 10, 14, 12, 15],
        [ 0,  6,  6,  1,  2,  7,  5,  7,  8, 14, 14,  9, 10, 15, 13, 15],
        [ 3,  3,  3,  1,  3,  7,  5,  3, 11, 11, 11,  9, 11, 15, 13, 11]],
       dtype=int64))

It can be proved that SER 2 finds the sparsest possible decomposition for $P_6(0.02)$.

In [16]:
PBGDM_PBN_P6_d2 = np.zeros((16, 16), dtype=int)
PBGDM_PBN_P6_d2[0:8, 0:8] = PBGDM_PBN_P4_d2
PBGDM_PBN_P6_d2[8:16, 8:16] = PBGDM_PBN_P4_d2

SER_2(PBGDM_PBN_P6_d2)

(5,
 [15, 15, 10, 9, 1],
 array([[ 0,  6,  6,  4,  2,  6,  4,  7,  8, 14, 14, 12, 10, 14, 12, 15],
        [ 0,  6,  6,  5,  2,  7,  5,  7,  8, 14, 14, 13, 10, 15, 13, 15],
        [ 0,  6,  6,  0,  2,  6,  4,  7,  8, 14, 14,  8, 10, 14, 12, 15],
        [ 0,  6,  6,  1,  2,  7,  5,  7,  8, 14, 14,  9, 10, 15, 13, 15],
        [ 3,  3,  3,  1,  3,  7,  5,  3, 11, 11, 11,  9, 11, 15, 13, 11]],
       dtype=int64))

It can be proved that SER 2 finds the sparsest possible decomposition for $P_6(0.03)$.

In [17]:
PBGDM_PBN_P6_d3 = np.zeros((16, 16), dtype=int)
PBGDM_PBN_P6_d3[0:8, 0:8] = PBGDM_PBN_P4_d3
PBGDM_PBN_P6_d3[8:16, 8:16] = PBGDM_PBN_P4_d3

SER_2(PBGDM_PBN_P6_d3)

(5,
 [30, 30, 20, 17, 3],
 array([[ 0,  6,  6,  4,  2,  6,  4,  7,  8, 14, 14, 12, 10, 14, 12, 15],
        [ 0,  6,  6,  5,  2,  7,  5,  7,  8, 14, 14, 13, 10, 15, 13, 15],
        [ 0,  6,  6,  0,  2,  6,  4,  7,  8, 14, 14,  8, 10, 14, 12, 15],
        [ 0,  6,  6,  1,  2,  7,  5,  7,  8, 14, 14,  9, 10, 15, 13, 15],
        [ 3,  3,  3,  1,  3,  7,  5,  3, 11, 11, 11,  9, 11, 15, 13, 11]],
       dtype=int64))

It can be proved that SER 2 finds the sparsest possible decomposition for $P_6(0.04)$.

In [18]:
PBGDM_PBN_P6_d4 = np.zeros((16, 16), dtype=int)
PBGDM_PBN_P6_d4[0:8, 0:8] = PBGDM_PBN_P4_d4
PBGDM_PBN_P6_d4[8:16, 8:16] = PBGDM_PBN_P4_d4

SER_2(PBGDM_PBN_P6_d4)

(5,
 [15, 15, 10, 8, 2],
 array([[ 0,  6,  6,  4,  2,  6,  4,  7,  8, 14, 14, 12, 10, 14, 12, 15],
        [ 0,  6,  6,  5,  2,  7,  5,  7,  8, 14, 14, 13, 10, 15, 13, 15],
        [ 0,  6,  6,  0,  2,  6,  4,  7,  8, 14, 14,  8, 10, 14, 12, 15],
        [ 0,  6,  6,  1,  2,  7,  5,  7,  8, 14, 14,  9, 10, 15, 13, 15],
        [ 3,  3,  3,  1,  3,  7,  5,  3, 11, 11, 11,  9, 11, 15, 13, 11]],
       dtype=int64))